## Instructions

In this lab you'll be doing a bit more of a comprehensive analysis. Despite coming from Kaggle, this will not be a competition. Check out the data source and context here:

https://www.kaggle.com/datasets/riinuanslan/sleep-data-from-fitbit-trackerLinks to an external site.

It's a relatively simple dataset, but your goal is this: 

Can we predict Sleep Score (posted by the FitBit app) using the other metrics in the dataset? In other words, is there a formula here that the FitBit app uses to compute Sleep Score that we can reverse-engineer?
Two constraints for this assignment:

1. Your modeling efforts must involve bagging and stacking in some way. Otherwise, you may try whatever you like.

2. You are allowed, even encouraged, to compute and/or gather additional features to use as explanatory variables in your model. For example, you might create a variable for the time they went to sleep (as a measure of how "early" they went to bed, or not). There are multiple datasets and you should use all of them, which means you may use the corresponding month for the dataset as a variable as well (or anything related to it).

Your submission should be an HTML or .ipynb file of all of your work.

## Data Cleaning and Preparation

After some quick examination, I noticed a few typos that needed to be fixed. 

In [51]:
import pandas as pd

In [52]:
# File paths
file_paths = [
    "Data/November Sleep Data - Sheet1.csv",
    "Data/December Sleep data - Sheet1.csv",
    "Data/January sleep data - Sheet1.csv",
    "Data/February sleep data - Sheet1 (1).csv",
    "Data/March sleep data - Sheet1.csv",
    "Data/April sleep data - Sheet1.csv"
]

# Function to clean each file
def clean_sleep_df(file_path, month_name):
    df = pd.read_csv(file_path)

    # Drop first row if mostly empty
    if df.iloc[0].isnull().sum() > len(df.columns) // 2 or (df.iloc[0] == df.columns).sum() > len(df.columns) // 2:
        df = df.iloc[1:]

    # Standardize column names
    df.columns = df.columns.str.replace(month_name.upper(), 'Day of Week', regex=False)
    df.columns = df.columns.str.strip().str.upper()

    # Rename if columns exist
    col_renames = {
        'SLEEP SQORE': 'SLEEP SCORE',
        'HEART RATE UNDER RESTING': 'HEART RATE BELOW RESTING',
        'HEARTRATE BELOW RESTING': 'HEART RATE BELOW RESTING'
    }
    df = df.rename(columns={col: new for col, new in col_renames.items() if col in df.columns})

    return df

# Map of months to file paths, February is special!
month_map = {
    "November": file_paths[0],
    "December": file_paths[1],
    "January": file_paths[2],
    "Febeuary": file_paths[3],
    "March": file_paths[4],
    "April": file_paths[5],
}

# Clean and combine all DataFrames
df_all = pd.concat(
    [clean_sleep_df(path, month) for month, path in month_map.items()],
    ignore_index=True
)

df_all.head()

,DAY OF WEEK,DATE,SLEEP SCORE,HOURS OF SLEEP,REM SLEEP,DEEP SLEEP,HEART RATE BELOW RESTING,SLEEP TIME
0,Monday,11/1/2021,88.0,8:06:00,20.00%,13.00%,84.00%,10:41pm - 7:54am
1,Tuesday,11/2/2021,83.0,7:57:00,12.00%,18.00%,90.00%,10:40pm - 7:55am
2,Wednesday,11/3/2021,81.0,7:06:00,13.00%,22.00%,93.00%,11:03pm - 7:16am
3,Thursday,11/4/2021,86.0,7:04:00,19.00%,17.00%,97.00%,10:55pm - 6:56am
4,Friday,11/5/2021,81.0,9:24:00,17.00%,15.00%,66.00%,10:14pm - 9:01am


### Clean up Sleep Time

In [53]:
# Split 'SLEEP TIME' into 'BED TIME' and 'WAKE UP TIME'
df_all[['BED TIME', 'WAKE UP TIME']] = df_all['SLEEP TIME'].astype(str).str.split(' - ', expand=True)
df_all = df_all.drop(columns=['SLEEP TIME'])

# Check rows where values didn't parse cleanly
print(df_all[df_all['WAKE UP TIME'].str.endswith('m') == False])
print(df_all[df_all['BED TIME'].str.endswith('m') == False])

   DAY OF WEEK        DATE  SLEEP SCORE HOURS OF SLEEP REM SLEEP DEEP SLEEP  \
37   Wednesday  12/08/2021         85.0        6:47:00    22.00%     16.00%   
57     Tuesday  12/28/2021         85.0        6:57:00    20.00%     18.00%   

   HEART RATE BELOW RESTING BED TIME WAKE UP TIME  
37                   94.00%  11:40pm         7:33  
57                   99.00%  11:16pm         7:02  
    DAY OF WEEK        DATE  SLEEP SCORE HOURS OF SLEEP REM SLEEP DEEP SLEEP  \
5      Saturday   11/6/2021         77.0        8:19:00    14.00%     13.00%   
125      Sunday  03/06/2022          NaN            NaN       NaN        NaN   
138    Saturday  03/19/2022          NaN            NaN       NaN        NaN   

    HEART RATE BELOW RESTING BED TIME WAKE UP TIME  
5                     21.00%    11:21       8:45am  
125                      NaN      nan         None  
138                      NaN      nan         None  


### Feature Engineering and Further Cleaning

In [54]:
df_all = df_all.dropna(subset=['BED TIME', 'WAKE UP TIME'])
# Add 'pm' to BED TIME if it doesn’t already end in 'am' or 'pm'
df_all['BED TIME'] = df_all['BED TIME'].astype(str)
df_all.loc[~df_all['BED TIME'].str.endswith(('am', 'pm')), 'BED TIME'] += 'pm'

# Add 'am' to WAKE UP TIME if it doesn’t already end in 'am' or 'pm'
df_all['WAKE UP TIME'] = df_all['WAKE UP TIME'].astype(str)
df_all.loc[~df_all['WAKE UP TIME'].str.endswith(('am', 'pm')), 'WAKE UP TIME'] += 'am'

# Fix common typo in both time columns
df_all['BED TIME'] = df_all['BED TIME'].str.replace('-', ':', regex=False)
df_all['WAKE UP TIME'] = df_all['WAKE UP TIME'].str.replace('-', ':', regex=False)

In [55]:
# Fix DATE column and extract components
df_all['DATE'] = pd.to_datetime(df_all['DATE'], errors='coerce')
df_all['YEAR'] = df_all['DATE'].dt.year
df_all['MONTH'] = df_all['DATE'].dt.month
df_all['DAY'] = df_all['DATE'].dt.day
df_all = df_all.drop(columns=['DATE'])

# Convert percentages to decimals
pct_cols = ['REM SLEEP', 'DEEP SLEEP', 'HEART RATE BELOW RESTING']
for col in pct_cols:
    df_all[col] = df_all[col].astype(str).str.replace('%', '', regex=False).astype(float) / 100

# Convert HOURS OF SLEEP (hh:mm:ss) to float hours
def time_to_hours(t):
    try:
        parts = [int(x) for x in str(t).split(':')]
        return parts[0] + parts[1]/60 + (parts[2]/3600 if len(parts) == 3 else 0)
    except:
        return None

df_all['HOURS OF SLEEP'] = df_all['HOURS OF SLEEP'].apply(time_to_hours)

# Categorize times of day
def categorize_time(t):
    try:
        time = pd.to_datetime(t, format="%I:%M%p").time()
        hour = time.hour + time.minute / 60
        if hour < 4:
            return "Late Night"
        elif hour < 8:
            return "Early Morning"
        elif hour < 12:
            return "Morning"
        elif hour < 17:
            return "Afternoon"
        elif hour < 21:
            return "Evening"
        else:
            return "Night"
    except:
        return "Unknown"

df_all['BED TIME CATEGORY'] = df_all['BED TIME'].apply(categorize_time)
df_all['WAKE TIME CATEGORY'] = df_all['WAKE UP TIME'].apply(categorize_time)

def extract_hour_float(t):
    try:
        dt = pd.to_datetime(t, format="%I:%M%p")
        return dt.hour + dt.minute / 60
    except:
        return None

df_all['BED TIME (HOUR)'] = df_all['BED TIME'].apply(extract_hour_float)
df_all['WAKE UP TIME (HOUR)'] = df_all['WAKE UP TIME'].apply(extract_hour_float)

df_all.drop(columns = ['BED TIME', 'WAKE UP TIME'], inplace=True)

In [56]:
df_sleep_clean = df_all.copy()
df_sleep_clean.head()

,DAY OF WEEK,SLEEP SCORE,HOURS OF SLEEP,REM SLEEP,DEEP SLEEP,HEART RATE BELOW RESTING,YEAR,MONTH,DAY,BED TIME CATEGORY,WAKE TIME CATEGORY,BED TIME (HOUR),WAKE UP TIME (HOUR)
0,Monday,88.0,8.100000,0.20,0.13,0.84,2021,11,1,Night,Early Morning,22.683333,7.900000
1,Tuesday,83.0,7.950000,0.12,0.18,0.90,2021,11,2,Night,Early Morning,22.666667,7.916667
2,Wednesday,81.0,7.100000,0.13,0.22,0.93,2021,11,3,Night,Early Morning,23.050000,7.266667
3,Thursday,86.0,7.066667,0.19,0.17,0.97,2021,11,4,Night,Early Morning,22.916667,6.933333
4,Friday,81.0,9.400000,0.17,0.15,0.66,2021,11,5,Night,Morning,22.233333,9.016667


### Save to CSV for future use...

In [57]:
df_sleep_clean.to_csv("Data/Cleaned_Sleep_Data.csv")